In [1]:
import pandas as pd
import numpy as np

import librosa

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.math import confusion_matrix
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from tqdm import tqdm

2022-03-01 10:12:28.957738: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-01 10:12:28.957792: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def load_data(n):
    targets = pd.read_csv('targets.csv')
    group = targets.groupby('gender_emotion')
    sample = group.head(n)
    return targets

In [3]:
def cut_or_pad(array, n):
    if array.shape[1] < n:
        return pad_sequences(array,n,padding='post',value=-1000.)
    elif array.shape[1] > n:
        return array[:,:n]
    else:
        return array

In [4]:
def extract_features(df,
                     sr=44100,
                     length=250,
                     offset=0.3,
                     n_mfcc=10,
                     poly_order=4,
                     n_chroma=10,
                     n_mels=10):
    
    features = []
    
    for path in tqdm(df['path']):
        wav = librosa.load(path, sr=sr, offset=offset, duration = 10)
        
        mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
        mfcc = cut_or_pad(mfcc,length)
        
        poly = librosa.feature.poly_features(y=wav[0],sr=sr,order=poly_order)
        poly = cut_or_pad(poly,length)
        
        chroma = librosa.feature.chroma_stft(y=wav[0],sr=sr,n_chroma=n_chroma)
        chroma = cut_or_pad(chroma,length)
    
        melspec = librosa.feature.melspectrogram(y=wav[0],sr=sr,n_mels=n_mels)
        melspec = cut_or_pad(melspec,length)
        log_S = librosa.amplitude_to_db(melspec)
    
#         zcr = librosa.feature.zero_crossing_rate(y=wav[0])
#         zcr = cut_or_pad(zcr,length)
        
#         rms = librosa.feature.rms(y=wav[0])
#         rms = cut_or_pad(rms,length)
    
    #     f_tempo = librosa.feature.fourier_tempogram(y=wav[0], sr=sr, win_length=18)
    #     f_tempo = cut_or_pad(f_tempo,length)
        
    #     tempo = librosa.feature.tempogram(y=wav[0], sr=SAMPLE_RATE, win_length=10)
    #     tempo = cut_or_pad(tempo,length)
        
    #     spectral_centroid = librosa.feature.spectral_centroid(y=wav[0],sr=sr)
    #     spectral_centroid = cut_or_pad(spectral_centroid,length)
        
    #     spectral_bandwidth = librosa.feature.spectral_bandwidth(y=wav[0],sr=sr)
    #     spectral_bandwidth = cut_or_pad(spectral_bandwidth,length)
        
    #     spectral_contrast = librosa.feature.spectral_contrast(y=wav[0],sr=sr)
    #     spectral_contrast = cut_or_pad(spectral_contrast,length)
        
    #     spectral_flatness = librosa.feature.spectral_flatness(y=wav[0])
    #     spectral_flatness = cut_or_pad(spectral_flatness,length)
        
    #     spectral_rolloff = librosa.feature.spectral_rolloff(y=wav[0])
    #     spectral_rolloff = cut_or_pad(spectral_rolloff,length)
        
    #     tonnetz = librosa.feature.tonnetz(y=wav[0],sr=sr)
    #     tonnetz = cut_or_pad(tonnetz,length)
        
    #     chroma_cqt = librosa.feature.chroma_cqt(y=wav[0],sr=sr,n_chroma=10,bins_per_octave=30)
    #     chroma_cqt = cut_or_pad(chroma_cqt,length)
        
    #     chroma_cens = librosa.feature.chroma_cens(y=wav[0],sr=sr,n_chroma=10,bins_per_octave=30)
    #     chroma_cens = cut_or_pad(chroma_cens,length)
    
    
        #combine
        array = np.vstack((mfcc,
                           poly,
                           chroma,
                           log_S,
#                            zcr,
#                            rms,
    #                        f_tempo,
    #                        tempo,
    #                        spectral_centroid,
    #                        spectral_bandwidth,
    #                        spectral_contrast ,
    #                        spectral_flatness ,
    #                        spectral_rolloff  ,
    #                        tonnetz           ,
    #                        chroma_cqt        ,
    #                        chroma_cens,
                      ))
    
        array = array.T

        #mfcc = np.concatenate((mfcc,poly),axis=0,dtype=None)
        features.append(array)
    
    X = pad_sequences(features, dtype='float32', padding='post', value=-1000.)
    return X

In [18]:
def Xy(df,array, test_size=0.2):
    y = df['pos_or_neg']
    le = LabelEncoder()
    y_num = le.fit_transform(y)
    y_cat = to_categorical(y_num)
    
    X_train, X_val, y_train, y_val = train_test_split(array, y_cat, test_size=test_size, random_state=0)
    
    return X_train, X_val, y_train, y_val

In [19]:
def make_model(X):
    model = Sequential()
    model.add(layers.Masking(mask_value = -1000., input_shape=(X.shape[1], X.shape[2])))
    model.add(layers.LSTM(units=X_train.shape[2], return_sequences=True, activation='tanh'))
    model.add(layers.LSTM(64, return_sequences=False, activation='tanh'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(20, activation='relu'))
    #model.add(layers.Dropout(0.1))
    model.add(layers.Dense(15, activation='relu'))
    #model.add(layers.Dropout(0.1))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(6, activation="softmax"))
    
    optimiser = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimiser,metrics='acc')
    
    model.summary()
    return model

In [7]:
data = load_data(225)

In [8]:
X = extract_features(data,
                     sr=44100,
                     length=260,
                     offset=0.2,
                     n_mfcc=12,
                     poly_order=3,
                     n_chroma=10,
                     n_mels=10)

  0%|                                                                                          | 0/2700 [00:00<?, ?it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-6.6712564e-03 -7.6333415e-03 -7.3763202e-03 ...  9.9765680e-07
  3.1930197e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  0%|                                                                                  | 1/2700 [00:00<29:40,  1.52it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[ 2.6515196e-03  3.0636524e-03  2.9433763e-03 ...  6.4893459e-07
 -2.2089328e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  0%|                                                                                  | 2/2700 [00:00<17:55,  2.51it/s]/tmp/ipyke

  1%|▌                                                                                | 19/2700 [00:03<07:04,  6.32it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[ 6.8049799e-03  7.5738193e-03  7.0328098e-03 ... -2.9809482e-06
 -2.3183886e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  1%|▌                                                                                | 20/2700 [00:03<07:13,  6.19it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[0.00344809 0.00388625 0.00369164 ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  1%|▋                                                                                | 21/2700 [00:04<07:13,  6.17it/s]/tmp/ipykernel_19348/3196483463.py:

  1%|█▏                                                                               | 39/2700 [00:07<08:12,  5.40it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-1.4199197e-02 -1.6325003e-02 -1.6029138e-02 ... -5.6605835e-08
  1.3802916e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  1%|█▏                                                                               | 40/2700 [00:07<08:18,  5.34it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[0.00412379 0.00490092 0.00508746 ... 0.00023626 0.00015272 0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  2%|█▏                                                                               | 41/2700 [00:07<08:39,  5.12it/s]/tmp/ipykernel_19348/3196483463.py:

  2%|█▋                                                                               | 58/2700 [00:10<07:49,  5.62it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-5.5537028e-03 -6.5087322e-03 -6.5174405e-03 ... -4.2894180e-06
  1.1485255e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  2%|█▊                                                                               | 59/2700 [00:11<07:48,  5.64it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[2.5521487e-03 2.9708305e-03 2.9972354e-03 ... 2.9341913e-05 2.2261349e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  2%|█▊                                                                               | 60/2700 [00:11<07:57,  5.53it/s]/tmp/ipykernel_1

  3%|██▎                                                                              | 77/2700 [00:14<07:41,  5.69it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-2.8098836e-03 -3.0663570e-03 -2.9042100e-03 ...  1.2984621e-08
  7.5259088e-10  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  3%|██▎                                                                              | 78/2700 [00:14<07:54,  5.53it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[ 4.3090782e-03  5.0142305e-03  4.9758344e-03 ... -5.8791520e-06
  1.6771075e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  3%|██▎                                                                              | 79/2700 [00:14<07:47,  5.60it/s]/tmp/ipyke

...

 97%|████████████████████████████████████████████████████████████████████████████▌  | 2618/2700 [08:38<00:16,  4.86it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-0.00204537 -0.0020149  -0.00177452 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 97%|████████████████████████████████████████████████████████████████████████████▋  | 2619/2700 [08:39<00:16,  4.89it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-0.00584195 -0.00670177 -0.00642387 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 97%|████████████████████████████████████████████████████████████████████████████▋  | 2620/2700 [08:39<00:16,  4.99it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[

 98%|█████████████████████████████████████████████████████████████████████████████▏ | 2638/2700 [08:42<00:12,  4.97it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-0.0026351  -0.0030361  -0.00280456 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 98%|█████████████████████████████████████████████████████████████████████████████▏ | 2639/2700 [08:43<00:12,  4.82it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-4.0420182e-03 -4.2691366e-03 -3.7018587e-03 ... -5.8590834e-09
  3.9184762e-09  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 98%|█████████████████████████████████████████████████████████████████████████████▏ | 2640/2700 [08:43<00:12,  4.85it/s]/tmp/ipykernel_19348/3196483463.py:1

 98%|█████████████████████████████████████████████████████████████████████████████▊ | 2658/2700 [08:46<00:07,  5.27it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[0.00696629 0.00799334 0.00790649 ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 98%|█████████████████████████████████████████████████████████████████████████████▊ | 2659/2700 [08:46<00:07,  5.22it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[5.7224277e-03 6.0734907e-03 5.2786451e-03 ... 1.5108164e-08 1.9482778e-08
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 99%|█████████████████████████████████████████████████████████████████████████████▊ | 2660/2700 [08:47<00:07,  5.18it/s]/tmp/ipykernel_19348/3196483463.py:15: Fu

 99%|██████████████████████████████████████████████████████████████████████████████▎| 2677/2700 [08:50<00:04,  5.22it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[2.5474918e-03 3.1700607e-03 3.4685975e-03 ... 9.1660273e-05 4.0932373e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 99%|██████████████████████████████████████████████████████████████████████████████▎| 2678/2700 [08:50<00:04,  5.25it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-0.00310621 -0.00397918 -0.00431711 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 99%|██████████████████████████████████████████████████████████████████████████████▍| 2679/2700 [08:50<00:03,  5.59it/s]/tmp/ipykernel_19348/3196483463.py:15: Fut

100%|██████████████████████████████████████████████████████████████████████████████▉| 2697/2700 [08:54<00:00,  4.91it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-0.0043135  -0.00465808 -0.00436694 ...  0.0001998   0.00013585
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
100%|██████████████████████████████████████████████████████████████████████████████▉| 2698/2700 [08:54<00:00,  4.87it/s]/tmp/ipykernel_19348/3196483463.py:15: FutureWarning: Pass y=[-8.1191612e-03 -9.1850953e-03 -8.5946321e-03 ...  2.1450282e-05
  9.3147592e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
100%|██████████████████████████████████████████████████████████████████████████████▉| 2699/2700 [08:54<00:00,  4.94it/s]/tmp/ipykernel_19348/3196483

In [14]:
X_train, X_val, y_train, y_val = Xy(data,X)

In [10]:
model = make_model(X_train)

2022-03-01 10:21:26.490495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-01 10:21:26.490581: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-01 10:21:26.490640: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-LF7KQEFR): /proc/driver/nvidia/version does not exist
2022-03-01 10:21:26.491048: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 260, 36)           0         
                                                                 
 lstm (LSTM)                 (None, 260, 36)           10512     
                                                                 
 lstm_1 (LSTM)               (None, 64)                25856     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 20)                660       
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_2 (Dense)             (None, 15)                3

In [11]:
es = EarlyStopping(patience=20,monitor='val_loss')

model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=es,
         batch_size=32,
         epochs=100, verbose=1)

Epoch 1/100


2022-03-01 10:21:27.717710: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 80870400 exceeds 10% of free system memory.


68/68 [==============================] - 31s 376ms/step - loss: 1.7790 - acc: 0.1773 - val_loss: 1.7221 - val_acc: 0.2407
Epoch 2/100
68/68 [==============================] - 24s 356ms/step - loss: 1.7722 - acc: 0.2278 - val_loss: 1.7832 - val_acc: 0.1815
Epoch 3/100
68/68 [==============================] - 24s 352ms/step - loss: 1.7761 - acc: 0.2065 - val_loss: 1.7503 - val_acc: 0.2093
Epoch 4/100
68/68 [==============================] - 24s 354ms/step - loss: 1.7075 - acc: 0.2750 - val_loss: 1.6434 - val_acc: 0.2889
Epoch 5/100
68/68 [==============================] - 24s 355ms/step - loss: 1.6534 - acc: 0.2880 - val_loss: 1.6228 - val_acc: 0.3111
Epoch 6/100
68/68 [==============================] - 24s 353ms/step - loss: 1.6466 - acc: 0.2884 - val_loss: 1.6340 - val_acc: 0.3019
Epoch 7/100
68/68 [==============================] - 24s 356ms/step - loss: 1.6138 - acc: 0.3148 - val_loss: 1.5791 - val_acc: 0.3463
Epoch 8/100
68/68 [==============================] - 25s 363ms/step - loss

Epoch 62/100
68/68 [==============================] - 24s 359ms/step - loss: 1.4286 - acc: 0.4329 - val_loss: 1.4356 - val_acc: 0.4315
Epoch 63/100
68/68 [==============================] - 24s 359ms/step - loss: 1.4451 - acc: 0.4116 - val_loss: 1.4604 - val_acc: 0.4259
Epoch 64/100
68/68 [==============================] - 24s 360ms/step - loss: 1.5522 - acc: 0.3662 - val_loss: 1.5546 - val_acc: 0.3407
Epoch 65/100
68/68 [==============================] - 24s 357ms/step - loss: 1.4851 - acc: 0.3940 - val_loss: 1.4679 - val_acc: 0.4204
Epoch 66/100
68/68 [==============================] - 24s 358ms/step - loss: 1.4781 - acc: 0.4134 - val_loss: 1.5176 - val_acc: 0.4056
Epoch 67/100
68/68 [==============================] - 24s 359ms/step - loss: 1.4594 - acc: 0.4162 - val_loss: 1.4922 - val_acc: 0.4241
Epoch 68/100
68/68 [==============================] - 24s 358ms/step - loss: 1.4526 - acc: 0.4218 - val_loss: 1.5148 - val_acc: 0.4296
Epoch 69/100
68/68 [==============================] - 2